In [1]:
import numpy as np
import time
import json

In [2]:

def alpha2F(om): #quadratic approximation for the phonon spectrum
    return om**2

def pairbreakingdistributions(E,om,Delta): #distribution for the pair breaking interaction
    omminE = np.round(om-E,5)
    dist =  1/(E**2-Delta**2)**0.5   *(E*(omminE)+Delta**2)/((omminE)**2-Delta**2)**0.5
    return dist


def phonon_emiss_dist(om,E,spectrum,Delta): #distribution for the phonon emission interaction
    dist = spectrum(om)*(E-om)/((E-om)**2-Delta**2)**0.5   *(1-Delta**2/(E*(E-om)))
    return dist



#DISCRETIZATION
#These fucntions calculate the possible outcomes of all interactions and corresponding chances.
#The outcomes are indexes in the energy-array, which used as an input to define what energies are possible in the simulation

def descretize_init_phon(spectrum,energy): #the initial phonon spectrum

    om_allowed = energy
    outcomes = np.arange(len(om_allowed))
    chances = spectrum(om_allowed)/sum(spectrum(om_allowed))
    print("Finished initial phonon descretisation")
    return outcomes, chances

def descretize_pair_breaking(energy,stepsize,Delta): #the pair breaking interaction
    om_allowed  =energy
    outcomes = []
    chances = []
    loc_delta = round(1/stepsize)
    indexarray = np.arange(0,len(energy)+1,1)
    for i in range(len(om_allowed)):
        indexes = indexarray[loc_delta:i-loc_delta+1] #define the allowed range of the outcomes
        outcomes.append(indexes)
        E = energy[indexes]
        chance = pairbreakingdistributions(E,om_allowed[i],Delta) #calculate the value of the probability distribution
        if len(chance) > 1:
            chance[-1] = pairbreakingdistributions(E[-1]-stepsize/4,om_allowed[i],Delta)/2 #the last bin needs to be halved in order to prevent an infinite chance, see report
        norm_chance = chance/sum(chance) #normalise the chances
        chances.append(norm_chance)
    print("finished pair breaking descretisation")
    return outcomes, chances

def descretize_phonon_emiss(spectrum,energy,stepsize,Delta):
    E_allowed = energy

    outcomes = []
    chances = []

    indexarray = np.arange(0,len(energy)+1,1)
    loc_delta = round(1/stepsize)

    for i in range(len(E_allowed)):
        indexes = indexarray[0:(i+1)-loc_delta] #define the allowed range of the outcomes
        outcomes.append(indexes)

        om = energy[indexes]
        chance = phonon_emiss_dist(om,E_allowed[i],spectrum,Delta) #calculate the value of the probability distribution
        if len(chance) > 1:
                chance[-1] = phonon_emiss_dist(om[-1]-stepsize/4,E_allowed[i],spectrum,Delta)/2  #the last bin needs to be halved in order to prevent an infinite chance, see report
        norm_chance = chance/sum(chance) #normalise the chances
        chances.append(norm_chance)

    print("finished phonon emission descretisation")
    return outcomes, chances

In [3]:
def pick_init_phon(Q,init_phon,energy,stepsize): #function to initialise the first group of phonons with energy equal to Q, the photon energy
    outcomes,chances = init_phon
    phonons = np.zeros(len(energy))

    while sum(phonons*energy) < Q:
        index = np.random.choice(outcomes,p=chances)
        phonons[index] += 1
    phonons[index] += -1
    totalenergy = np.sum(phonons*energy)

    lastenergy = int(Q-totalenergy)
    lastindex = int(lastenergy/stepsize)
    phonons[lastindex] += 1
    return phonons


def pick_quasis(index,pairbreaking,energy):
    outcomes, chances = pairbreaking
    try:
        out = outcomes[index]
        probs = chances[index]

        energy1 = np.random.choice(out,p=probs)
        energy2 = index-energy1
    except:
        print(energy[index])
        print(energy[outcomes[index]])
        print(outcomes[index],chances[index])
    quasis  = np.array([energy1,energy2])

    diff = energy[energy1]+energy[energy2]-energy[index]
    if np.abs(diff) > 1/10:
        print("no conservation:", diff,'Delta deviation')
        print(index,energy1,energy2)

    return quasis.astype(int)


def pick_phonon(index,emission,energy):
    outcomes, chances = emission[0], emission[1]
    try:
        phonon = np.random.choice(outcomes[index],p=chances[index])
    except:
        print(energy[index])
        print(outcomes[index],chances[index])
    return phonon.astype(int)


#main simulation loop
def simulation(Q,energy,cycles,stepsize,pairbreaking,emission,init_phon,control):
    start_time = time.time()
    N = np.zeros(cycles)
    ph = np.zeros(cycles)
    efficiency = np.zeros(cycles)
    E_mean = np.zeros(cycles)
    om_mean = np.zeros(cycles)

    L = len(energy)
    qlimit = round(3/stepsize) #limits where energies of phonons and quasiparticle cannot cause the creation of more quasiparticles
    plimit = round(2/stepsize)
    pairbreaking[1][plimit] = [1]
    #at a phonon energy 2*Delta, the discretisation does not function properly due to the fact that there is just one outcome (two Delta quasparticles)
    #possible and this results in infinite probability, which is why we need to set in manually

    for i in range(cycles):
        #some text to make the interface more clear
        print("\n ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~") #to separate iterations
        print("Cycle number: {}".format(i+1))
        #initialise the first phonons and an empty list for the quasiparticles
        quasiparticles = np.zeros(L)
        phonons = np.zeros(L)
        initphonons = (pick_init_phon(Q,init_phon,energy,stepsize))
        phonons += initphonons

        #cycle that repeats until all interacting particles are exhausted
        while sum(phonons[plimit:])>0 or sum(quasiparticles[qlimit:])>0:

            for p in range(plimit,L):
                if phonons[p] > 0:
                    for j in range(int(phonons[p])):
                        phonons[p] += -1
                        indexes = pick_quasis(p,pairbreaking,energy)
                        quasiparticles[indexes] += 1

            for q in range(qlimit,L):
                if quasiparticles[q] > 0:
                    for j in range(int(quasiparticles[q])):
                        #print(quasiparticles[q])
                        quasiparticles[q] += -1
                        index = pick_phonon(q,emission,energy)
                        phonons[index] += 1
                        newquasis = q - index
                        quasiparticles[newquasis] += 1



        #the sum of the quasiparticle array gives the no. of quasiparticles
        N[i]= sum(quasiparticles)
        ph[i] = sum(phonons)
        efficiency[i]= (sum(quasiparticles)*(1+stepsize/2)/Q)
        E_mean[i] = sum(quasiparticles*energy)/sum(quasiparticles)
        om_mean[i] = sum(phonons*energy)/sum(phonons)

        energycontrol = sum(phonons*energy)+sum(quasiparticles*energy)
        stop_time = time.time()
        Delta_t = stop_time-start_time
        print("Time passed: {} s".format(round(Delta_t,2)))
        if control == True: #print control variables if True
            print("Efficiency: {} ".format(efficiency[i]))
            print("Number of quasiparticles: {}".format(N[i]))
            print("Total energy: {} \u0394".format(round(energycontrol),3))
            print("Phonon energy: {} \u0394".format(round(sum(phonons*energy))))
            print("Quasiparticle energy: {} \u0394".format(round(sum(quasiparticles*energy))))
            print("Mean phonon energy: {} \u0394".format(round(om_mean[i],2)))
            print("Mean quasiparticle energy: {} \u0394".format(round(E_mean[i],2)))
            print("Energy deficiency: {} \u0394".format(sum(initphonons*energy)-energycontrol))

    return N, ph, efficiency, E_mean,om_mean

#SAVING DATA
def savedata(data,name,spec,cycles,wd):
    filename = name+str(spec)+"_"+"wd"+str(int(wd))+"_"+"2024-1-3"+"_"+str(cycles)+"its"
    data = list(data)
    with open("Data\\"+filename, 'w') as file:
        json.dump(data, file)


In [4]:
#CONSTANTS
Delta = 1 #because we calculate in units of Delta
h = 4.136e-15 #eV s. planck constant
kb = 8.617343e-5 #eV K^-1, Boltzmann constant

#Material
realDelta , wd = 667e-6, 240 # Ta
# realDelta,wd= 5.75e-4, 200# eV,K Gap energy and debye temperature of Sn
# realDelta, wd= 15.25e-4, 276 # eV Gap energy and debye temperature of Nb
# realDelta, wd= 1.7e-4, 433 #eV,K gap energy and debye temperature of Al
#realDelta, wd = 0.75e-4, 209 #eV, K, gap energy and debye temperatue of Cd

#general allowed energies of E and Omega
Debye = wd*kb/realDelta


#Simulation parameters
cycles = 50 #number of iterations
spectrum = alpha2F   #phonon spectrum used
control = True #whether or not control variables are printed

stepsize = [1/200]
Q = [10000]
spectra = [alpha2F]

#we use lists to be able to quickly switch between different imput parameters
for j in range(len(spectra)):
    for p in range(len(Q)):
        for i in range(len(stepsize)):

            #initialising and discretisation
            start = stepsize[i]/2
            energy  = np.arange(start,Debye+start,stepsize[i])

            tstart = time.time()
            init_phon = descretize_init_phon(spectra[j],energy)
            pairbreaking = descretize_pair_breaking(energy,stepsize[i],Delta)
            emission = descretize_phonon_emiss(spectra[j],energy,stepsize[i],Delta,)
            print("Finished discretisation in {} s".format(np.round(time.time()-tstart,2)))

            #execute simulation
            N, ph, efficiency, E_mean, om_mean = simulation(Q[p],energy,cycles,stepsize[i],pairbreaking,emission,init_phon,control)

            spec = spectra[j].__name__ #give your data a unique name so it is not overwritten

            #saving some data, make sure you have a folder named "data"
            savedata(N,'N',spec,cycles,wd)
            savedata(ph,'ph',spec,cycles,wd)
            savedata(efficiency,'eff',spec,cycles,wd)
            savedata(E_mean,'E',spec,cycles,wd)
            savedata(om_mean,'om',spec,cycles,wd)

Finished initial phonon descretisation


/tmp/ipykernel_57842/1668801092.py:6: RuntimeWarning: divide by zero encountered in divide
  dist =  1/(E**2-Delta**2)**0.5   *(E*(omminE)+Delta**2)/((omminE)**2-Delta**2)**0.5
/tmp/ipykernel_57842/1668801092.py:41: RuntimeWarning: invalid value encountered in divide
  norm_chance = chance/sum(chance) #normalise the chances
/tmp/ipykernel_57842/1668801092.py:6: RuntimeWarning: invalid value encountered in sqrt
  dist =  1/(E**2-Delta**2)**0.5   *(E*(omminE)+Delta**2)/((omminE)**2-Delta**2)**0.5


finished pair breaking descretisation


/tmp/ipykernel_57842/1668801092.py:11: RuntimeWarning: invalid value encountered in sqrt
  dist = spectrum(om)*(E-om)/((E-om)**2-Delta**2)**0.5   *(1-Delta**2/(E*(E-om)))
/tmp/ipykernel_57842/1668801092.py:11: RuntimeWarning: divide by zero encountered in divide
  dist = spectrum(om)*(E-om)/((E-om)**2-Delta**2)**0.5   *(1-Delta**2/(E*(E-om)))
/tmp/ipykernel_57842/1668801092.py:63: RuntimeWarning: invalid value encountered in divide
  norm_chance = chance/sum(chance) #normalise the chances


finished phonon emission descretisation
Finished discretisation in 3.7 s

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Cycle number: 1
Time passed: 0.55 s
Efficiency: 0.5912745 
Number of quasiparticles: 5898.0
Total energy: 10004 Δ
Phonon energy: 707 Δ
Quasiparticle energy: 9297 Δ
Mean phonon energy: 1.52 Δ
Mean quasiparticle energy: 1.58 Δ
Energy deficiency: -4.200000000004366 Δ

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Cycle number: 2
Time passed: 1.07 s
Efficiency: 0.5925777499999999 
Number of quasiparticles: 5911.0
Total energy: 9994 Δ
Phonon energy: 689 Δ
Quasiparticle energy: 9305 Δ
Mean phonon energy: 1.53 Δ
Mean quasiparticle energy: 1.57 Δ
Energy deficiency: 4.937499999996362 Δ

 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Cycle number: 3
Time passed: 1.61 s
Efficiency: 0.5944825 
Number of quasiparticles: 5930.0
Total energy: 10005 Δ
Phonon energy: 658 Δ
Quasiparticle energy: 9347 Δ
Mean phonon energy: 1.51 Δ
Mean quasiparticle energy: 1.58 Δ
Energy deficiency: -6.28000000000975 Δ

 ~~~~~~~~~~~~~